In [1]:
import pandas as pd
import numpy as np
import lib_tools as pt # Project Tools

# Complete
start_year, end_year, chk, sampled = [2005, 2021, True, False]

# Quick
# start_year, end_year, chk, sampled = [2005, 2021, True, True]
#
df = pt.load_proj_df(start_year, end_year, chk, sampled)
#
df.to_csv(path_or_buf='project_df.csv')


somme des lignes 'dic': 2509620
nombre de lignes 'df' : 2509620
nombre de doublons avant traîtement : 2858
nombre de doublons avant traîtement : 0
somme des lignes 'dic': 1121571
nombre de lignes 'df' : 1121571
nombre de doublons avant traîtement : 0
nombre de doublons avant traîtement : 0
somme des lignes 'dic': 1914902
nombre de lignes 'df' : 1914902
nombre de doublons avant traîtement : 0
nombre de doublons avant traîtement : 0
Véhicules en doublons vis à vis de la clé fonctionnelle avant traitement : 53
Véhicules en doublons vis à vis de la clé fonctionnelle après traitement : 0
somme des lignes 'dic': 1121571
nombre de lignes 'df' : 1121571
nombre de doublons avant traîtement : 0
nombre de doublons avant traîtement : 0

usagers : 

usagers 2005 : 12 colonnes x 197498 lignes
usagers 2006 : 12 colonnes x 187085 lignes
usagers 2007 : 12 colonnes x 188457 lignes
usagers 2008 : 12 colonnes x 170960 lignes
usagers 2009 : 12 colonnes x 165962 lignes
usagers 2010 : 12 colonnes x 154192 li

In [2]:
df = pd.read_csv('project_df.csv', index_col=0)
# df.head()

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_12152\2965180363.py:1: DtypeWarning: Columns (7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('project_df.csv', index_col=0)


In [3]:
data = df.iloc[:, 1:]
target = df['grav']
data = data.join(pd.get_dummies(data['joursem'], prefix='joursem'))
data = data.join(pd.get_dummies(data['actp'], prefix='actp'))
data = data.join(pd.get_dummies(data['dep'], prefix='dep'))
data = data.drop(columns=['joursem'], axis=1)
data = data.drop(columns=['actp'], axis=1)
data = data.drop(columns=['dep'], axis=1)
# data = data.drop(columns=['hrmn'], axis=1)

KeyError: "['hrmn'] not found in axis"

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

dt_clf = DecisionTreeClassifier(criterion ='entropy', max_depth=4, random_state=123)
dt_clf.fit(X_train, y_train)

y_pred = dt_clf.predict(X_test)

pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,1,3,4
Classe réelle,,,
1,177688,758,26541
2,7239,2644,3522
3,43585,12726,43902
4,90271,6929,84453


In [4]:
data

,place,catu,sexe,trajet,locp,etatp,mois,jour,lum,agg,...,dep_972,dep_973,dep_974,dep_975,dep_976,dep_977,dep_978,dep_986,dep_987,dep_988
0,1,1,1,1,0,0,1,12,3,2,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,3,0,0,1,12,3,2,...,0,0,0,0,0,0,0,0,0,0
2,2,2,1,0,0,0,1,12,3,2,...,0,0,0,0,0,0,0,0,0,0
3,4,2,1,0,0,0,1,12,3,2,...,0,0,0,0,0,0,0,0,0,0
4,5,2,1,0,0,0,1,12,3,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506757,1,1,1,0,0,-1,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
2506758,1,1,1,5,0,-1,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
2506759,1,1,1,0,0,-1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2506760,1,1,1,3,-1,-1,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
